# An Introduction to WISER, Part 3: Neural Networks

In this part of the tutorial, we use the probabilistic labels of the generative model to train a state-of-the-art recurrent neural network. We will use a bi-LSTM-CRF to improve the generalization of the link-HMM

To begin, we will enable logging to keep track of the training logs. You may observe this in the terminal used to open this notebook.

In [1]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

## Training a Neural Network

To train your bi-LSTM-CRF network, you first need to define an [AllenNLP configuration file](https://github.com/allenai/allennlp/blob/master/tutorials/getting_started/walk_through_allennlp/configuration.md). These files provide a versatile way of defining neural networks, detailing all the model parameters. For this tutorial, we have already provided a configuration file, but feel free to edit it or use your own

We have provided the wrapper method ``train_discriminative_model`` on ``wiser.discriminative`` that you can use to . 
To train a bi-LSTM-CRF on the link HMM's probabilistic output, you need to call the method and pass it in the location of the generative models' output. You may also specify the cuda device and random seed for training. 

During training, tou should be able to observe logs on the terminal used to open this notebook. TODO: Mention that they shouldn't expect an evolution of train scores

In [ ]:
from wiser.discriminative import train_discriminative_model

train_discriminative_model(
    train_data_path='output/generative/link_hmm/train_data.p',
    dev_data_path='output/generative/link_hmm/dev_data.p',
    test_data_path='output/generative/link_hmm/test_data.p',
    train_config_path='training_config/tutorial.jsonnet',
    output_path='output/discriminative/link_hmm',
    cuda_device=-1)

## Predicting Tags

In [ ]:
# TODO